# AWS Transcribe

Transcribe an audio file to text.

<a href="https://docs.aws.amazon.com/transcribe/index.html#lang/en_us">AWS Transcribe Documentation</a>

This notebook uploads an audio file to S3, transcribes it, and then deletes the file from S3.

In [1]:
import boto3
import time
import json
import urllib.request

In [2]:
def aws_s3_upload(file_name, bucket_name):
    s3 = boto3.resource('s3')
    
    # Create bucket if it doesn't already exist
    bucket_names = [b.name for b in s3.buckets.all()]
    if bucket_name not in bucket_names:
        s3.create_bucket(Bucket=bucket_name,
                         CreateBucketConfiguration={'LocationConstraint': 'EU'})
        print("Bucket {} created.".format(bucket_name))
        
    s3.meta.client.upload_file(file_name, bucket_name, file_name)
    print("{} uploaded to {}.".format(file_name, bucket_name))
    return


def aws_s3_delete(file_name, bucket_name, del_bucket=False):
    s3 = boto3.resource('s3')
    try:
        s3.meta.client.delete_object(Bucket=bucket_name, Key=file_name)
        print("{} deleted from {}.".format(file_name, bucket_name))
    except:
        print("Unable to delete {} from {}.".format(file_name, bucket_name))
        
    if del_bucket:
        try:
            s3.meta.client.delete_bucket(Bucket=bucket_name)
            print("Bucket {} deleted.".format(bucket_name))
        except:
            print("Unable to delete bucket {}.".format(bucket_name))    
    return

In [3]:
def aws_transcribe(file_name, bucket_name):
    client = boto3.client(service_name='transcribe',
                          region_name='eu-west-1',
                          use_ssl=True)
    job_name = 'example_%s' % round(time.time())
    job_uri = 's3://%s/%s' % (bucket_name, file_name)
    client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': job_uri},
        MediaFormat=file_name[-3:],
        LanguageCode='en-US'
    )
    tic = time.time()
    while True:
        status = client.get_transcription_job(TranscriptionJobName=job_name)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        toc = time.time()
        print("Transcription still processing... cumulative run time: {:.1f}s".format(toc-tic))
        time.sleep(10)
    print("Transcription completed! Total run time: {:.1f}s".format(toc-tic))
    
    json_url = status['TranscriptionJob']['Transcript']['TranscriptFileUri']
    with urllib.request.urlopen(json_url) as url:
        text = json.loads(url.read().decode())
    return text['results']['transcripts'][0]['transcript']

In [4]:
file_name = 'the_raven.mp3'
bucket_name = 'your_bucket_name'

In [5]:
aws_s3_upload(file_name, bucket_name)
result = aws_transcribe(file_name, bucket_name)
aws_s3_delete(file_name, bucket_name)

the_raven.mp3 uploaded to your_bucket_name.
Transcription still processing... cumulative run time: 0.1s
Transcription still processing... cumulative run time: 10.1s
Transcription still processing... cumulative run time: 20.2s
Transcription still processing... cumulative run time: 30.3s
Transcription still processing... cumulative run time: 40.5s
Transcription still processing... cumulative run time: 50.6s
Transcription still processing... cumulative run time: 60.7s
Transcription still processing... cumulative run time: 70.7s
Transcription still processing... cumulative run time: 80.8s
Transcription completed! Total run time: 80.8s
the_raven.mp3 deleted from your_bucket_name.


In [6]:
# Print transcription
print(result)

Once upon a midnight dreary, while i pondered, weak and weary over many a quaint and curious volume of forgotten law, while i nodded, nearly napping. Suddenly there came a tapping as of someone gently rapping, rapping at my chamber door. To some visitor, i muttered, tapping at my chamber door. Only this and nothing more.
